In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!ls

gdrive	sample_data


In [3]:
%cd /content/gdrive/MyDrive/CS271P_project

/content/gdrive/MyDrive/CS271P_project


In [4]:
!ls

1m_tweets.csv  final_dataset.csv  glove


In [5]:
import pandas as pd
import re
import string
from wordcloud import WordCloud
from textblob import TextBlob, Word
import matplotlib.pyplot as plt
import nltk
nltk.download('wordnet')
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

tweets = pd.read_csv('final_dataset.csv', encoding="utf-8")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [6]:
def remove_unnecessary_data_from(tweets):
    refined_tweets = []
    for tweet in tweets:
        tweet = str(tweet)
        # Converting to lowercase
        tweet = tweet.lower()
        # Remove unnecessary things like mentions and hashtags
        tweet = re.sub('@[^ ]+|#[^ ]+', '', tweet)
        # Remove punctuation
        tweet = tweet.replace('[^A-Za-z0-9 ]', "")
        # Lemmatize all the words
        tweet = " ".join([Word(word).lemmatize() for word in tweet.split()])
        refined_tweets.append(tweet)
    return refined_tweets

In [7]:
X = remove_unnecessary_data_from([tweet for tweet in tweets['text']])

In [8]:
x_train, x_test, y_train, y_test = train_test_split(X, tweets.label, test_size=0.2, random_state = 42)

In [9]:
val_len = len(x_test)//2
x_val = x_test[:val_len]
y_val = y_test[:val_len]
x_test = x_test[val_len:]
y_test = y_test[val_len:]

In [10]:
logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=1, C=1e5)),
               ])
logreg.fit(x_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(C=100000.0, n_jobs=1))])

In [11]:
y_pred = logreg.predict(x_val)

print('accuracy %s' % accuracy_score(y_pred, y_val))
print(classification_report(y_val, y_pred, digits=5))

accuracy 0.9955022488755623
              precision    recall  f1-score   support

           0    0.99127   1.00000   0.99561       681
           1    1.00000   0.99081   0.99538       653

    accuracy                        0.99550      1334
   macro avg    0.99563   0.99541   0.99550      1334
weighted avg    0.99554   0.99550   0.99550      1334

